In [ ]:
from itertools import permutations
import numpy as np
import copy
from tqdm import tqdm
import pandas as pd
from pandas import DataFrame 
import os
from math import pi

import bokeh
from bokeh.io import output_file, show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, ColumnDataSource, FixedTicker, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.transform import transform
from bokeh.layouts import gridplot

from bokeh.models import (BoxZoomTool, Circle, HoverTool,
                          MultiLine, Plot, Range1d, ResetTool,)
from bokeh.palettes import Spectral4
from bokeh.palettes import Viridis256
from bokeh.plotting import from_networkx

import networkx as nx
import community



import colorcet
cmap = colorcet.glasbey_dark


from sklearn.neighbors import NearestNeighbors

bokeh.io.output_notebook()

In [ ]:
def needleman_wunsch(seq1, seq2, gap):
    '''
    Code to determine the needleman wunsch distance between two sequences, using the bolsum_50 matrix.
    '''
    amino_acid_dict = {'A':0, 'R':1, 'N':2, 'D':3, 'C':4, 'Q':5, 'E':6, 'G':7, 'H':8, 'I':9,  'L':10, 'K':11, 'M':12, 'F':13, 'P':14, 'S':15, 'T':16, 'W':17, 'Y':18, 'V':19}
    
    blosum50 = [[ 5,-2, -1, -2, -1, -1, -1,  0, -2, -1, -2, -1, -1, -3, -1,  1,  0, -3, -2,  0, -2],
            [-2, 7, -1, -2, -4,  1,  0, -3,  0, -4, -3,  3, -2, -3, -3, -1, -1, -3, -1, -3, -1],
            [-1,-1,  7,  2, -2,  0,  0,  0,  1, -3, -4,  0, -2, -4, -2,  1,  0, -4, -2, -3,  5],
            [-2,-2,  2,  8, -4,  0,  2, -1, -1, -4, -4, -1, -4, -5, -1,  0, -1, -5, -3, -4,  6],
            [-1,-4, -2, -4, 13, -3, -3, -3, -3, -2, -2, -3, -2, -2, -4, -1, -1, -5, -3, -1, -3],
            [-1, 1,  0,  0, -3,  7,  2, -2,  1, -3, -2,  2,  0, -4, -1,  0, -1, -1, -1, -3,  0],
            [-1, 0,  0,  2, -3,  2,  6, -3,  0, -4, -3,  1, -2, -3, -1, -1, -1, -3, -2, -3,  1],
            [ 0,-3,  0, -1, -3, -2, -3,  8, -2, -4, -4, -2, -3, -4, -2,  0, -2, -3, -3, -4, -1],
            [-2, 0,  1, -1, -3,  1,  0, -2, 10, -4, -3,  0, -1, -1, -2, -1, -2, -3,  2, -4,  0],
            [-1,-4, -3, -4, -2, -3, -4, -4, -4,  5,  2, -3,  2,  0, -3, -3, -1, -3, -1,  4, -4],
            [-2,-3, -4, -4, -2, -2, -3, -4, -3,  2,  5, -3,  3,  1, -4, -3, -1, -2, -1,  1, -4],
            [-1, 3,  0, -1, -3,  2,  1, -2,  0, -3, -3,  6, -2, -4, -1,  0, -1, -3, -2, -3,  0],
            [-1,-2, -2, -4, -2,  0, -2, -3, -1,  2,  3, -2,  7,  0, -3, -2, -1, -1,  0,  1, -3],
            [-3,-3, -4, -5, -2, -4, -3, -4, -1,  0,  1, -4,  0,  8, -4, -3, -2,  1,  4, -1, -4],
            [-1,-3, -2, -1, -4, -1, -1, -2, -2, -3, -4, -1, -3, -4, 10, -1, -1, -4, -3, -3, -2],
            [ 1,-1,  1,  0, -1,  0, -1,  0, -1, -3, -3,  0, -2, -3, -1,  5,  2, -4, -2, -2,  0],
            [ 0,-1,  0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1,  2,  5, -3, -2,  0,  0],
            [-3,-3, -4, -5, -5, -1, -3, -3, -3, -3, -2, -3, -1,  1, -4, -4, -3, 15,  2, -3, -5],
            [-2,-1, -2, -3, -3, -1, -2, -3,  2, -1, -1, -2,  0,  4, -3, -2, -2,  2,  8, -1, -3],
            [ 0,-3, -3, -4, -1, -3, -3, -4, -4,  4,  1, -3,  1, -1, -3, -2,  0, -3, -1,  5, -3]]
    
    # Create an alignment matrix to store match and mismatch values
    alignment_matrix = np.zeros((len(seq1) + 1, len(seq2) + 1))
        
    # Create an position matrix to store what positions the current value came from
    position_matrix = np.zeros((len(seq1) + 1, len(seq2) + 1))
    
    # Set the boundary conditions for the matrix
    for i in range(len(seq1) + 1):
        alignment_matrix[i, 0] += i*gap
    
    for j in range(len(seq2) + 1):
        alignment_matrix[0, j] += j*gap
        if j > 0:
            position_matrix[0, j] += 1
    
    # Loop through the matrix
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):            
            value = blosum50[amino_acid_dict[seq1[i-1]]][amino_acid_dict[seq2[j-1]]]
                
            match_list = [alignment_matrix[i-1, j] + gap, alignment_matrix[i, j-1] + gap, alignment_matrix[i-1, j-1] + value]
            
            inds = np.argmax(match_list)
            
            alignment_matrix[i, j] += match_list[inds] 
        
            position_matrix[i, j] = inds
    
    seq1_alignment = []
    seq2_alignment = []
    
    n = len(seq1)
    m = len(seq2)
    
    while (n or m) > 0:
        val = position_matrix[n,m]
    
        if val == 2:
            seq1_alignment = [seq1[n-1]] + seq1_alignment 
            seq2_alignment = [seq2[m-1]] + seq2_alignment
            
            n = n - 1
            m = m - 1
            
        elif val == 0:
            seq1_alignment = [seq1[n-1]] + seq1_alignment 
            seq2_alignment = ['_'] + seq2_alignment
            n -= 1
            
        elif val == 1:
            seq1_alignment = ['_'] + seq1_alignment
            seq2_alignment = [seq2[m-1]] + seq2_alignment
            
        
            m -= 1
    
    seq1_str = ''.join(seq1_alignment)
    seq2_str = ''.join(seq2_alignment)
        
    return(seq1_str, seq2_str, alignment_matrix[-1, -1])

In [ ]:
### Edit to be your filepath
fname = 'Priya_R2_DNAandvirus_over10_Tekbrain_over50_nodup_nospikein_B.csv'

# Could convert to csv if needed
df = pd.read_csv(fname, comment = '#')

#top50 variants
df = df.nlargest(50, 'Tek-mean-synth')

#Alternatively, you could select ositively enriched variants
#df = df.loc[(df['Tek-mean-synth']>0)]

In [ ]:
#the number of variants you have in your dataset, scales NxN so should be kept small.
n = len(df)

alignment_scores = np.empty((n, n))
alignment_scores.fill(np.nan)

# There is some nuance here... The [2:-2] is to remove amino acids that are invariable from the sequences. This may need to be adjusted based on the data you're working with. -8 is the gap penalty, which allows amino acids that are in different locations to be aligned with a penalty
for i in tqdm(range(n)):
    alignment_scores[i,i] = needleman_wunsch(df['Amino Acid'][i][2:-2],df['Amino Acid'][i][2:-2],-8)[2]
    
    for j in range(i, n):
        alignment_scores[i,j] = (needleman_wunsch(df['Amino Acid'][i][2:-2],df['Amino Acid'][j][2:-2],-8)[2])
         
        alignment_scores[j,i] =  alignment_scores[i,j] 

In [ ]:
# Initialize dataframes for normalized scores
normalized_alignment_scores = np.empty((n, n))
normalized_alignment_scores.fill(np.nan)

In [ ]:
# Normalize the scores
for i in range(n):
    normalized_alignment_scores[i] = (alignment_scores[i] - np.nanmin(alignment_scores[i])) / (alignment_scores[i,i] - np.nanmin(alignment_scores[i]))

normalized_alignment_scores[-1, -1] = 1

In [ ]:
# Create a dataframe for plotting
normalized_alignment_df = pd.DataFrame(
    normalized_alignment_scores,
    columns=df['Amino Acid'][0:len(df)],
    index=df['Amino Acid'][0:len(df)])
normalized_alignment_df.index.name = 'source'
normalized_alignment_df.columns.name = 'target'

In [ ]:
# Prepare data.frame in the right format
stacked_normalized_alignment_df = normalized_alignment_df.stack().rename("value").reset_index()

In [ ]:
# Plot the data

mapper = LinearColorMapper(
    palette='Viridis256', low=stacked_normalized_alignment_df['value'].min(), high=stacked_normalized_alignment_df['value'].max())

# Define a figureabsabs
p = figure(
    plot_width=1000,
    plot_height=1000,
    x_range=list(stacked_normalized_alignment_df['source'].drop_duplicates()),
    y_range=list(stacked_normalized_alignment_df['target'].drop_duplicates()),
    title="Positively enriched variants from CNS (mouse)",
    toolbar_location='right',
    tools = "pan,wheel_zoom,box_zoom,reset,save",
    x_axis_location="above")

# Create rectangle for heatmap
p.rect(
    x="source",
    y="target",
    width=1,
    height=1,
    source=ColumnDataSource(stacked_normalized_alignment_df),
    line_color=None,
    fill_color=transform('value', mapper))

# Add legend
color_bar = ColorBar(
    color_mapper=mapper,
    location=(0, 0))

p.add_layout(color_bar, 'right')
p.y_range.range_padding = 0.005
p.x_range.range_padding = 0.005

p.xaxis.major_label_orientation = pi/4

show(p)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(normalized_alignment_scores)
distances, indices = nbrs.kneighbors(normalized_alignment_scores)

A = nbrs.kneighbors_graph(normalized_alignment_scores).toarray()

G = nx.from_numpy_matrix(A)
partition = community.best_partition(G)

In [ ]:
#version without the labels
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = 12,8

pos = nx.spring_layout(G, k=0.01)

communities = []

for i, com in enumerate(set(partition.values())):
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    
    #print(df['Amino Acid'][list_nodes][:])
    nx.draw_networkx_nodes(G, 
                           pos, 
                           list_nodes,
                           node_size=200,
                           node_color=cmap[i])

nx.draw_networkx_labels(G, pos, labels = df['Rank'], font_size=12, font_color='w', font_weight='bold')
